# SOLID Principles

## Single Responsability Principle

A class with many responsabilities is as useless as a class with none.

*How to recognize a violation to SRP?*
 - Class has many instance variables
 - Class has many public methods
 - Each method uses different instance variables
 - Specific tasks are delegated to private methods

In [1]:
# Base code that we will be using in many examples

import random
from dataclasses import dataclass


@dataclass
class User:
    
    __email: str

    @property
    def email(self) -> str:
        return self.__email


@dataclass
class Message:
    
    subject: str
    body: str
      
    @property
    def to(self) -> str:
        return self.__to

    @to.setter
    def to(self, value: str):
        self.__to = value


class Templater:
    
    def render(self, filename, values):
        return " ".join(values.values())


class Translator:
    
    def translate(self, msg):
        return msg


class Mailer:
    
    def send(self, message: Message):
        print("To: {to}\nSubject: {subject}\nBody: {body}".format(
            to=message.to, subject=message.subject, body=message.body
        ))


In [2]:
# Bad example with many violations to SRP

class ConfirmationMailer:
    
    def __init__(self, templater, translator, mailer):
        # Each of the components implements an specific interface
        self.__templater = templater
        self.__translator = translator
        self.__mailer = mailer

    def send_to(self, user: User) -> bool:
        message = self._create_message_for(user)
        self._send_message(message)


    def _create_message_for(self, user: User):
        subject = self.__translator.translate("Confirm your email")
        body = self.__templater.render("confirmation.html", {
            "code": str(random.randint(1000, 9999))
        })
        message = Message(subject, body)
        message.to = user.email
        return message

    def _send_message(self, message: Message):
        self.__mailer.send(message)
    

In [3]:
# It actually works but with many caveats
c = ConfirmationMailer(Templater(), Translator(), Mailer())
c.send_to(User("aldo@me.com"))

To: aldo@me.com
Subject: Confirm your email
Body: 8106


The class above has actually more than just one responsability, we are delegating the task of creating an email and then send it to an user.

That is enough reason to rewrite this class trying to minimize the number of resposabilities, this would at the same time reduce the need of modify the class in the future.

We already know that ConfirmationMailer does too many things, refactoring this should be quite straightforward, since is a class Mailer, we'll delegate the task of sending emails to it, yet we are going to substract the responsability of create the message.

Create a message is not as easy as instanciate a new Message object, it requires a few dependencies, therefore we might need a dedicated *Factory* class.

In [4]:
class Factory:
    pass


class MailFactory(Factory):

    def __init__(self, templater, translator):
        self.__templater = templater
        self.__translator = translator

    def create_message_for(self, user):
        subject = self.__translator.translate("Confirm your email")
        body = self.__templater.render("confirmation.html", {
            "code": str(random.randint(1000, 9999))
        })
        message = Message(subject, body)
        message.to = user.email
        return message


class RefactoredConfirmationMailer:

    def __init__(self, factory, mailer):
        self.__factory = factory
        self.__mailer = mailer
        
    def send_to(self, user):
        message = self._create_message_for(user)
        self._send_message(message)
    
    def _create_message_for(self, user):
        return self.__factory.create_message_for(user)

    def _send_message(self, message):
        self.__mailer.send(message)


We already extracted the task of creating a new email, 
with two classes with one responsability each one the code will be
easier to maintain, even if a differente factory is necessary
refactoring our code to use a different model now allows us to
inject any kind of Factory as long as they inherit from Factory base
class, which is empty now, but not forever.

In [5]:
rc = RefactoredConfirmationMailer(MailFactory(Templater(), Translator()), Mailer())
rc.send_to(User("aldo@me.com"))

To: aldo@me.com
Subject: Confirm your email
Body: 9734


As a side effect of this refactoring, now both classes are easier to test, now we might test both responsabilities individually, the correctness of a recently created email and the ability to send it to an user mocking the whoe message-creation process the test can be focused only on actually sending the email

### Conclusion

Any task represent a responsability inside a class, but responsabilities are also reasons to change the code, SRP is about minimizing the number of reasons for a class to be modified.

This usually mean the extraction of at least one collaborating classes, each of this will need a smaller number of dependencies, making each class easier to instantiate, test and use.

## The Open/Closed principle

The open/closed principle says that
    _You should be able to extend a class's behaviour without modifying it_
    
A piece of code can be considered "open for extension" when its behaviour can be extended without actually modifying it, the fact that to actual modification is needed to change the behaviour of any unit of code makes it "closed for modification".

In [6]:
# Base mock code that we will be using on this section

class JSONEncoder:
    
    def encode(self, data):
        return data


class XMLEncoder:
    
    def encode(self, data):
        return data
        

Take a look to the class below, a encoder with a method to decide which type of the previously defined encoders use

In [7]:
class BadEncoder:
    
    def encode_to_format(data, format_):
        if format_ == "json":
            encoder = JSONEncoder()
        elif format_ == "xml":
            encoder = XMLEncoder()
        else:
            raise RuntimeError(f"Invaluid encode format {format_}")
        
        return encoder.encode(data)


Let's assume we want to encode data using yaml format, which is currently not supported by our class, the obvious solution would be to create a YAMLEncoder for this and add a condition inside the `encode_to_format` function.

In [8]:
class YAMLEncoder:
    
    def encode(self, data):
        return data


class BadEncoder:
    
    def encode_to_format(data, format_):
        if format_ == "json":
            encoder = JSONEncoder()
        elif format_ == "xml":
            encoder = XMLEncoder()
        elif format_ == "yaml":
            encoder == YAMLEncoder()
        else:
            raise RuntimeError(f"Invaluid encode format {format_}")
        
        data = self.prepare_data(data, format_)
        return encoder.encode(data)

As you might have guessed, each time we need a new format for encoding we'll nedd to *modify* this conditional and create a new class, if tomorrow a new serializing method _puml_ is created, we will repeat the same process again and again. This is a clear violation to the open/closed principle, since we can not change the behaviour of the without modifiying the code

_How to recognize a violation to O/C P?_
 - Code contains conditions to determine a strategy
 - Conditions using the same variables or constants are recurring inside the class or related classes
 - Class contains hard-coded references to other classes or class names
 - Class is creating objects directly
 - The class has private properties or methods to allow changing its behaviour by overriding state
 

To fix this bad design which requires us to constantly dive into `BadEncoder`class, we'll need to delegate the responsability of resolving which class to use to another dedicated class, remember the first part of this guide, the logic for find a right encoder is an actual reason to change.

This new class might as well be an implementarion of the *Abstract Factory* design pattern, since Python does not need interfaces, we will be using a baseclass Encoder which will require an encode method, after all our Abstract Factory class does not care about what class is using, it is only interested on return an object that inherits from the base class.

In [9]:
class Encoder:
    
    def encode(self, data):
        pass
    

class JSONEncoder(Encoder):

    def encode(self, data):
        return data


class XMLEncoder(Encoder):

    def encode(self, data):
        return data


class YAMLEncoder(Encoder):
    
    def encode(self, data):
        return data
    

# Let's define our factory...

class EncoderFactory:
    
    # Returning type will be always an Encoder subclass
    def create_for_format(self, format_) -> Encoder:
        encoder = {
            "json": JSONEncoder,
            "xml": XMLEncoder,
            "yaml": YAMLEncoder,
        }.get(format_.lower())
        
        if encoder is None:
            raise RuntimeError(f"Invaluid encode format {format_}")
        return encoder()

Now we need to make sure that our main class does not create any other encoder, instead we delegated this task to`EncoderFactory` class, therefore if a new encoder is required, only the factory class will be modified.

In [10]:
class GoodEncoder:
    
    def __init__(self, factory):
        self.__factory = factory

    def encode_to_format(self, data, format_):
        encoder = self.__factory.create_for_format(data)
        encoder.encode(data)


Now our class also conforms the SRP, using the encoder factory for fetching the right encoder for a given format, we ensure that we won't need to modify the class `GoodEncoder` anymore, we'll need to modify the factory instead.

But, `EncoderFacotry` is still an ugly hard-coded list of supported formats and their encoders, this means that our factory is still closed against extension, it thereby violates the O/C P

Another refactor is required, this time we'll use the *Dependency inversion* design pattern, defining a 